# Tuberculosis Diagnosis using Transfer Learning
This notebook is written and executed by **Dr Raheel Siddiqi** on *05-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 5-fold cross validation is used to evaluate the model.

In [10]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.13.1
Keras Version:  2.2.4-tf


In [11]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model   

In [12]:
image_height = 100
image_width = 100
batch_size = 2
no_of_epochs  = 15

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
%matplotlib inline

DATADIR = "D:\\TensorFlow Programs\\TB Diagnosis\\ChinaSet_AllFiles\\ChinaSet_AllFiles\\ChinaSetImages"

CATEGORIES = ["NORMAL", "TB"]

data=[]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):  
        try:
            img_array = cv2.imread(os.path.join(path,img))
            img_array = cv2.resize(img_array, (image_height, image_width))
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
            img_array = img_array.astype(np.float32)/255.
            data.append([img_array, class_num])
        except Exception as e:   
            pass

print(len(data))

100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:30<00:00,  8.19it/s]


662


In [14]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

1
1
1
0
0
1
0
1
0
0


In [15]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


In [16]:
print(len(X))

662


In [17]:
print(X.shape)

(662, 100, 100, 3)


In [18]:
k=5
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=4,epochs=20) # 20 epochs per model
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])

Epoch 1/20
530/530 [==============================] - ETA: 1:31 - loss: 0.5506 - acc: 0.750 - ETA: 51s - loss: 0.5622 - acc: 0.750 - ETA: 38s - loss: 0.8537 - acc: 0.66 - ETA: 31s - loss: 0.8725 - acc: 0.50 - ETA: 27s - loss: 0.9967 - acc: 0.40 - ETA: 24s - loss: 0.9515 - acc: 0.37 - ETA: 22s - loss: 0.9163 - acc: 0.39 - ETA: 21s - loss: 0.8915 - acc: 0.37 - ETA: 19s - loss: 0.8701 - acc: 0.38 - ETA: 18s - loss: 0.8501 - acc: 0.40 - ETA: 18s - loss: 0.8271 - acc: 0.45 - ETA: 17s - loss: 0.8080 - acc: 0.47 - ETA: 16s - loss: 0.8293 - acc: 0.46 - ETA: 16s - loss: 0.8278 - acc: 0.46 - ETA: 15s - loss: 0.8240 - acc: 0.46 - ETA: 15s - loss: 0.8116 - acc: 0.48 - ETA: 14s - loss: 0.8026 - acc: 0.50 - ETA: 14s - loss: 0.7915 - acc: 0.52 - ETA: 14s - loss: 0.7832 - acc: 0.53 - ETA: 14s - loss: 0.7713 - acc: 0.56 - ETA: 13s - loss: 0.7605 - acc: 0.57 - ETA: 13s - loss: 0.7506 - acc: 0.59 - ETA: 13s - loss: 0.7491 - acc: 0.58 - ETA: 12s - loss: 0.7591 - acc: 0.58 - ETA: 12s - loss: 0.7507 - acc: 

530/530 [==============================] - ETA: 25s - loss: 0.1768 - acc: 1.00 - ETA: 21s - loss: 0.4753 - acc: 0.87 - ETA: 19s - loss: 0.5032 - acc: 0.83 - ETA: 17s - loss: 0.4438 - acc: 0.81 - ETA: 17s - loss: 0.4547 - acc: 0.80 - ETA: 16s - loss: 0.4247 - acc: 0.83 - ETA: 16s - loss: 0.4901 - acc: 0.78 - ETA: 15s - loss: 0.4844 - acc: 0.78 - ETA: 15s - loss: 0.4876 - acc: 0.77 - ETA: 15s - loss: 0.4489 - acc: 0.80 - ETA: 14s - loss: 0.4555 - acc: 0.79 - ETA: 14s - loss: 0.4448 - acc: 0.79 - ETA: 14s - loss: 0.4835 - acc: 0.76 - ETA: 13s - loss: 0.4498 - acc: 0.78 - ETA: 13s - loss: 0.4449 - acc: 0.80 - ETA: 13s - loss: 0.4353 - acc: 0.81 - ETA: 13s - loss: 0.4439 - acc: 0.80 - ETA: 12s - loss: 0.4309 - acc: 0.81 - ETA: 12s - loss: 0.4283 - acc: 0.82 - ETA: 12s - loss: 0.4346 - acc: 0.82 - ETA: 12s - loss: 0.4397 - acc: 0.80 - ETA: 11s - loss: 0.4541 - acc: 0.80 - ETA: 11s - loss: 0.4425 - acc: 0.81 - ETA: 11s - loss: 0.4302 - acc: 0.82 - ETA: 11s - loss: 0.4241 - acc: 0.83 - ETA: 11

530/530 [==============================] - ETA: 11s - loss: 0.9173 - acc: 0.75 - ETA: 11s - loss: 0.6349 - acc: 0.87 - ETA: 11s - loss: 0.5867 - acc: 0.83 - ETA: 11s - loss: 0.5423 - acc: 0.81 - ETA: 11s - loss: 0.5336 - acc: 0.80 - ETA: 11s - loss: 0.5963 - acc: 0.75 - ETA: 11s - loss: 0.5856 - acc: 0.75 - ETA: 11s - loss: 0.5391 - acc: 0.78 - ETA: 11s - loss: 0.5504 - acc: 0.75 - ETA: 11s - loss: 0.5677 - acc: 0.72 - ETA: 11s - loss: 0.5903 - acc: 0.70 - ETA: 11s - loss: 0.5817 - acc: 0.72 - ETA: 11s - loss: 0.5787 - acc: 0.73 - ETA: 11s - loss: 0.5629 - acc: 0.75 - ETA: 10s - loss: 0.5412 - acc: 0.76 - ETA: 10s - loss: 0.5117 - acc: 0.78 - ETA: 10s - loss: 0.5051 - acc: 0.77 - ETA: 10s - loss: 0.5155 - acc: 0.77 - ETA: 10s - loss: 0.6089 - acc: 0.73 - ETA: 10s - loss: 0.6153 - acc: 0.72 - ETA: 10s - loss: 0.6276 - acc: 0.72 - ETA: 10s - loss: 0.6109 - acc: 0.73 - ETA: 10s - loss: 0.5967 - acc: 0.75 - ETA: 10s - loss: 0.6234 - acc: 0.72 - ETA: 10s - loss: 0.6294 - acc: 0.73 - ETA: 9s

530/530 [==============================] - ETA: 11s - loss: 0.5480 - acc: 0.75 - ETA: 11s - loss: 0.4514 - acc: 0.87 - ETA: 11s - loss: 0.3840 - acc: 0.91 - ETA: 11s - loss: 0.3508 - acc: 0.93 - ETA: 11s - loss: 0.3902 - acc: 0.90 - ETA: 11s - loss: 0.3387 - acc: 0.91 - ETA: 11s - loss: 0.3150 - acc: 0.92 - ETA: 11s - loss: 0.2989 - acc: 0.93 - ETA: 11s - loss: 0.2889 - acc: 0.94 - ETA: 11s - loss: 0.3559 - acc: 0.90 - ETA: 11s - loss: 0.3394 - acc: 0.90 - ETA: 11s - loss: 0.3959 - acc: 0.87 - ETA: 11s - loss: 0.3809 - acc: 0.88 - ETA: 11s - loss: 0.4013 - acc: 0.87 - ETA: 10s - loss: 0.4278 - acc: 0.86 - ETA: 10s - loss: 0.4139 - acc: 0.87 - ETA: 10s - loss: 0.3977 - acc: 0.88 - ETA: 10s - loss: 0.4077 - acc: 0.87 - ETA: 10s - loss: 0.4411 - acc: 0.85 - ETA: 10s - loss: 0.4254 - acc: 0.86 - ETA: 10s - loss: 0.4134 - acc: 0.86 - ETA: 10s - loss: 0.4173 - acc: 0.86 - ETA: 10s - loss: 0.4033 - acc: 0.86 - ETA: 10s - loss: 0.3907 - acc: 0.87 - ETA: 10s - loss: 0.3988 - acc: 0.87 - ETA: 9s

530/530 [==============================] - ETA: 12s - loss: 0.5833 - acc: 0.75 - ETA: 12s - loss: 0.5836 - acc: 0.62 - ETA: 12s - loss: 0.4516 - acc: 0.75 - ETA: 11s - loss: 0.3835 - acc: 0.81 - ETA: 11s - loss: 0.4262 - acc: 0.80 - ETA: 11s - loss: 0.4441 - acc: 0.79 - ETA: 11s - loss: 0.4122 - acc: 0.82 - ETA: 11s - loss: 0.4110 - acc: 0.81 - ETA: 11s - loss: 0.3876 - acc: 0.83 - ETA: 11s - loss: 0.3880 - acc: 0.82 - ETA: 11s - loss: 0.3937 - acc: 0.81 - ETA: 11s - loss: 0.3743 - acc: 0.83 - ETA: 11s - loss: 0.3800 - acc: 0.82 - ETA: 11s - loss: 0.3824 - acc: 0.82 - ETA: 10s - loss: 0.3778 - acc: 0.83 - ETA: 10s - loss: 0.3693 - acc: 0.84 - ETA: 10s - loss: 0.3590 - acc: 0.85 - ETA: 10s - loss: 0.3606 - acc: 0.84 - ETA: 10s - loss: 0.3420 - acc: 0.85 - ETA: 10s - loss: 0.3450 - acc: 0.85 - ETA: 10s - loss: 0.3670 - acc: 0.83 - ETA: 10s - loss: 0.3607 - acc: 0.84 - ETA: 10s - loss: 0.4092 - acc: 0.83 - ETA: 10s - loss: 0.3967 - acc: 0.84 - ETA: 10s - loss: 0.3980 - acc: 0.84 - ETA: 9s

...

530/530 [==============================] - ETA: 12s - loss: 0.2281 - acc: 1.00 - ETA: 12s - loss: 0.1716 - acc: 1.00 - ETA: 12s - loss: 0.1450 - acc: 1.00 - ETA: 12s - loss: 0.1797 - acc: 0.93 - ETA: 11s - loss: 0.1986 - acc: 0.95 - ETA: 11s - loss: 0.1907 - acc: 0.95 - ETA: 11s - loss: 0.1896 - acc: 0.96 - ETA: 11s - loss: 0.1829 - acc: 0.96 - ETA: 11s - loss: 0.1714 - acc: 0.97 - ETA: 11s - loss: 0.2000 - acc: 0.95 - ETA: 11s - loss: 0.1914 - acc: 0.95 - ETA: 11s - loss: 0.1845 - acc: 0.95 - ETA: 11s - loss: 0.1739 - acc: 0.96 - ETA: 11s - loss: 0.2074 - acc: 0.94 - ETA: 11s - loss: 0.2017 - acc: 0.95 - ETA: 11s - loss: 0.1897 - acc: 0.95 - ETA: 10s - loss: 0.1801 - acc: 0.95 - ETA: 10s - loss: 0.1731 - acc: 0.95 - ETA: 10s - loss: 0.1641 - acc: 0.96 - ETA: 10s - loss: 0.1635 - acc: 0.96 - ETA: 10s - loss: 0.1580 - acc: 0.96 - ETA: 10s - loss: 0.1542 - acc: 0.96 - ETA: 10s - loss: 0.1489 - acc: 0.96 - ETA: 10s - loss: 0.1530 - acc: 0.95 - ETA: 10s - loss: 0.1481 - acc: 0.96 - ETA: 10

530/530 [==============================] - ETA: 12s - loss: 0.0138 - acc: 1.00 - ETA: 12s - loss: 0.0284 - acc: 1.00 - ETA: 12s - loss: 0.0352 - acc: 1.00 - ETA: 12s - loss: 0.0292 - acc: 1.00 - ETA: 12s - loss: 0.0547 - acc: 1.00 - ETA: 11s - loss: 0.0542 - acc: 1.00 - ETA: 11s - loss: 0.1175 - acc: 0.96 - ETA: 11s - loss: 0.1681 - acc: 0.93 - ETA: 11s - loss: 0.1633 - acc: 0.94 - ETA: 11s - loss: 0.1515 - acc: 0.95 - ETA: 11s - loss: 0.1503 - acc: 0.95 - ETA: 11s - loss: 0.1403 - acc: 0.95 - ETA: 11s - loss: 0.1388 - acc: 0.96 - ETA: 11s - loss: 0.1305 - acc: 0.96 - ETA: 11s - loss: 0.1257 - acc: 0.96 - ETA: 11s - loss: 0.1211 - acc: 0.96 - ETA: 10s - loss: 0.1393 - acc: 0.95 - ETA: 10s - loss: 0.1384 - acc: 0.95 - ETA: 10s - loss: 0.1322 - acc: 0.96 - ETA: 10s - loss: 0.1362 - acc: 0.96 - ETA: 10s - loss: 0.1376 - acc: 0.96 - ETA: 10s - loss: 0.1482 - acc: 0.95 - ETA: 10s - loss: 0.1419 - acc: 0.95 - ETA: 10s - loss: 0.1361 - acc: 0.95 - ETA: 10s - loss: 0.1314 - acc: 0.96 - ETA: 10

530/530 [==============================] - ETA: 12s - loss: 0.0339 - acc: 1.00 - ETA: 12s - loss: 0.0966 - acc: 1.00 - ETA: 12s - loss: 0.0729 - acc: 1.00 - ETA: 12s - loss: 0.0782 - acc: 1.00 - ETA: 11s - loss: 0.0667 - acc: 1.00 - ETA: 11s - loss: 0.0587 - acc: 1.00 - ETA: 11s - loss: 0.0591 - acc: 1.00 - ETA: 11s - loss: 0.0874 - acc: 0.96 - ETA: 11s - loss: 0.0801 - acc: 0.97 - ETA: 11s - loss: 0.0807 - acc: 0.97 - ETA: 11s - loss: 0.0750 - acc: 0.97 - ETA: 11s - loss: 0.0702 - acc: 0.97 - ETA: 11s - loss: 0.0766 - acc: 0.98 - ETA: 11s - loss: 0.0734 - acc: 0.98 - ETA: 11s - loss: 0.0688 - acc: 0.98 - ETA: 11s - loss: 0.0846 - acc: 0.96 - ETA: 10s - loss: 0.0805 - acc: 0.97 - ETA: 10s - loss: 0.0779 - acc: 0.97 - ETA: 10s - loss: 0.0743 - acc: 0.97 - ETA: 10s - loss: 0.0756 - acc: 0.97 - ETA: 10s - loss: 0.0742 - acc: 0.97 - ETA: 10s - loss: 0.0808 - acc: 0.96 - ETA: 10s - loss: 0.0776 - acc: 0.96 - ETA: 10s - loss: 0.0979 - acc: 0.95 - ETA: 10s - loss: 0.0944 - acc: 0.96 - ETA: 10

530/530 [==============================] - ETA: 12s - loss: 0.0203 - acc: 1.00 - ETA: 12s - loss: 0.1229 - acc: 0.87 - ETA: 12s - loss: 0.1247 - acc: 0.91 - ETA: 12s - loss: 0.0937 - acc: 0.93 - ETA: 12s - loss: 0.0755 - acc: 0.95 - ETA: 11s - loss: 0.0630 - acc: 0.95 - ETA: 11s - loss: 0.0541 - acc: 0.96 - ETA: 11s - loss: 0.0511 - acc: 0.96 - ETA: 11s - loss: 0.1002 - acc: 0.94 - ETA: 11s - loss: 0.0902 - acc: 0.95 - ETA: 11s - loss: 0.0844 - acc: 0.95 - ETA: 11s - loss: 0.0821 - acc: 0.95 - ETA: 11s - loss: 0.0793 - acc: 0.96 - ETA: 11s - loss: 0.0738 - acc: 0.96 - ETA: 11s - loss: 0.0689 - acc: 0.96 - ETA: 11s - loss: 0.0657 - acc: 0.96 - ETA: 10s - loss: 0.1020 - acc: 0.95 - ETA: 10s - loss: 0.1042 - acc: 0.95 - ETA: 10s - loss: 0.1018 - acc: 0.96 - ETA: 10s - loss: 0.1061 - acc: 0.95 - ETA: 10s - loss: 0.1979 - acc: 0.94 - ETA: 10s - loss: 0.1893 - acc: 0.94 - ETA: 10s - loss: 0.1824 - acc: 0.94 - ETA: 10s - loss: 0.1754 - acc: 0.94 - ETA: 10s - loss: 0.1698 - acc: 0.95 - ETA: 10

132/132 [==============================] - ETA: 1s - loss: 0.7426 - acc: 0.750 - ETA: 0s - loss: 0.7350 - acc: 0.781 - ETA: 0s - loss: 0.7225 - acc: 0.791 - ETA: 0s - loss: 0.6634 - acc: 0.796 - 1s 5ms/sample - loss: 0.6434 - acc: 0.8030


In [19]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.79393935
